## Instruction
### Installation of BLAST 
Install NCBI standalone BLAST from (https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=Download). \
Set the path to the BLAST.

### Make database
Download a fast file containing the transcriptome of your species. The example link is (http://useast.ensembl.org/info/data/ftp/index.html). \
The file is used to minimize the non-specific binding of oligonucleotides. \
Make a local database from the file according to (https://www.ncbi.nlm.nih.gov/books/NBK569841/). \
Example databases are included in ../data/ . Only database with ensemple transcript ID is accepted.

### Download 
Download the fasta file of mRNA of your interest from NCBI.
The example file is included in ../data/slc17a7/

In [1]:
import pandas as pd

# import sys
# sys.path.append('../oligodesigner')

from oligodesigner import blockParse_py3
from oligodesigner import io
from oligodesigner import parse
from oligodesigner import sequence
from oligodesigner import parameters

In [2]:
import importlib
importlib.reload(sequence)

<module 'oligodesigner.sequence' from '/home/tmurakami/src/split-oligo-designer/oligodesigner/sequence.py'>

In [3]:
# Configs
cdna_database = {
                 'human': './data/human_transcriptome/human_transcriptome_db',
                 'mouse': './data/mouse_transcriptome/mouse_transcriptome_db',
                 'rat'  : './data/rat_transcriptome/rat_transcriptome_db'
}


# hcr_b_set = {
#     'B1':{
#         'splitseq_ll':'GAGGAGGGCAGCAAACGG','splitseq_rr':'GAAGAGTCTTCCTTTACG','anchorseq_ll':'aa','anchorseq_rr':'at',
#         'splitseq_lr':'GGCAGCAAACGGGAAGAG','splitseq_rl':'GCATTCTTTCTTGAGGAG','anchorseq_lr':'tt','anchorseq_rl':'tt'
#     }
# }

In [13]:
# Parameter settings
mFISH3D_param = {
    'input_file':'/scratch1/tmurakami/dev_data/oligodesigner/slc17a7_hs_debug.fasta',#_debug
    'hcr_bnum':'B1',
    'cdna_database':cdna_database['human'],
    'minimum_offtarget_gap':100,
    # 'oligo_number': 150
}

oligominer_param = {
    'l':20,
    'L':20,
    'gcPercent':25,
    'GCPercent':75,
    'tm':0,
    'TM':100,
    'X':'AAAAAA,TTTTTT,CCCCCC,GGGGGG',
    'sal':390,#default value of blockParse
    'form':30,
    'sp':1,
    'concA':25,#default value of blockParse
    'concB':25,#default value of blockParse
    'headerVal':None,
    'bedVal':False,
    'OverlapModeVal':False,
    'verbocity':False,
    'reportVal':True,
    'debugVal':False,
    'metaVal':False,
    'outNameVal':None,
    'nn_table':'DNA_NN3'
}

In [15]:
# get transcripts that is the same gene product of the input

In [14]:
input_file = mFISH3D_param['input_file']
transcriptomeDatabase = mFISH3D_param['cdna_database']

self_remove = True 
# Default True. set self_remove True if you want to remove the input sequence from off-target analysis.
# Otherwise, the gene of your interest could be regarded as an off-target product.
sequence_homology_search = True
# If True, the search of the homology will be performed regardless of the existence of refseq id in fasta.

if self_remove:
    seqid = sequence.check_refseqid_exist(input_file, return_entrezid=True)
    if seqid is None:
        seqid = sequence.get_homology_in_database(input_file, transcriptomeDatabase, num_threads=32)
    elif sequence_homology_search:
        _ = sequence.get_homology_in_database(input_file, transcriptomeDatabase, num_threads=32)


no refseq id was found
The sequence is the most similar to SLC17A7 with the score of 0.6463361991574816
Homologous transcript(s) was identified. This may impact on the selection of the oligos
Homologous transcript SLC17A8 was found with the score of 0.1779060160120019


In [16]:
hcr_initiator_set = parameters.hcr_b_set[mFISH3D_param['hcr_bnum']]
minimum_offtarget_gap = mFISH3D_param['minimum_offtarget_gap']
# oligo_number = mFISH3D_param['oligo_number']

# Make path to temp files and result file
(oligominer_fastq, oligominer_fasta, blast_result, oligo_sets, param_file) = io.gen_file_path(input_file)
# Save parameters to csv file.
io.record_param(mFISH3D_param, oligominer_param, param_file)

In [17]:
# Run OligoMiner
blockParse_py3.runSequenceCrawler(input_file, *parse.oligominer_param_parser(oligominer_param))

0 of 2949
132 candidate probes identified in 2.94 kb yielding 44.96 candidates/kb


In [18]:
# converting fastq to fasta
io.convert_fastq2fasta(oligominer_fastq,oligominer_fasta)
io.add_id(oligominer_fasta)

# run blast
sequence.run_blast(oligominer_fasta,blast_result,transcriptomeDatabase)

In [19]:
# Read blast result file, exclude sequence which is homologous to the gene of interest from result
df_blast = sequence.exclude_self(io.read_blast(blast_result), seqid)

# Remove combination if sstart and send are close enough and if they are rective combination.
oligo_to_be_removed = sequence.get_off_targeting_oligo(minimum_offtarget_gap, df_blast)

In [20]:
oligominer_df = io.dataframe_from_oligominer(oligominer_fastq)
selected_oligo_df = sequence.remove_oligo(oligominer_df, oligo_to_be_removed)

interval = sequence.calc_oligo_interval(selected_oligo_df)
selected_oligo_df = pd.concat([selected_oligo_df,interval],axis=1)

# Make oligos with HCR sequences.
res = sequence.add_hcr_seq_v3(selected_oligo_df, hcr_initiator_set)
res.to_csv(oligo_sets, sep=',', index=False, header=True)

In [21]:
res

,gene,oligo_id,seq,start,end,interval_after,mean_interval,hcr_seq
0,NM_foomoo:1-20,NM_foomoo:1-20_0,"(A, C, T, T, G, C, A, G, C, C, T, C, C, T, T, ...",1,20,34.0,NaN,"(G, A, G, G, A, G, G, G, C, A, G, C, A, A, A, ..."
1,NM_foomoo:54-73,NM_foomoo:54-73_1,"(T, G, G, A, C, C, C, C, G, G, G, A, A, C, C, ...",54,73,59.0,NaN,"(G, C, A, T, T, C, T, T, T, C, T, T, G, A, G, ..."
2,NM_foomoo:132-151,NM_foomoo:132-151_2,"(C, G, G, C, A, G, G, A, G, C, C, G, C, C, A, ...",132,151,2.0,NaN,"(G, A, G, G, A, G, G, G, C, A, G, C, A, A, A, ..."
3,NM_foomoo:153-172,NM_foomoo:153-172_3,"(A, G, T, T, C, C, G, C, C, A, G, G, A, G, G, ...",153,172,2.0,NaN,"(G, C, A, T, T, C, T, T, T, C, T, T, G, A, G, ..."
4,NM_foomoo:174-193,NM_foomoo:174-193_4,"(G, G, A, A, G, C, T, A, G, C, G, G, G, T, C, ...",174,193,2.0,NaN,"(G, A, G, G, A, G, G, G, C, A, G, C, A, A, A, ..."
...,...,...,...,...,...,...,...,...
124,NM_foomoo:2798-2817,NM_foomoo:2798-2817_126,"(A, C, C, T, A, T, A, G, A, G, A, A, G, T, C, ...",2798,2817,2.0,NaN,"(G, A, G, G, A, G, G, G, C, A, G, C, A, A, A, ..."
125,NM_foomoo:2819-2838,NM_foomoo:2819-2838_127,"(T, C, C, C, A, T, G, A, A, A, G, G, G, A, A, ...",2819,2838,17.0,NaN,"(G, C, A, T, T, C, T, T, T, C, T, T, G, A, G, ..."
126,NM_foomoo:2855-2874,NM_foomoo:2855-2874_128,"(T, T, T, T, T, C, C, C, A, G, T, G, C, A, G, ...",2855,2874,23.0,NaN,"(G, A, G, G, A, G, G, G, C, A, G, C, A, A, A, ..."
127,NM_foomoo:2897-2916,NM_foomoo:2897-2916_130,"(T, G, A, C, T, G, T, A, A, A, T, C, C, T, G, ...",2897,2916,2.0,NaN,"(G, C, A, T, T, C, T, T, T, C, T, T, G, A, G, ..."
